# Web Scraping Project
Team Members: Joseph, Ryver, and Kari

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
# Define the URLs
base_url = 'https://www.mathjobs.org/jobs/list'
list_url = 'https://www.mathjobs.org/jobs?joblist-0-0------'

In [ ]:
# Get the job listing page
response = requests.get(list_url)
soup = bs(response.content, 'html.parser')

In [ ]:
# Get job listing containers
job_div_tag_list = soup.find_all('div', class_='clr')

In [ ]:
# Store date
data = {
    'Institution Name': [],
    'Position ID': [],
    'Position Title': [],
    'Position Location': [],
    'Subject Area': [],
    'Application Deadline': [],
    'Apply Link': [],
    'Job Details Link': []
}

In [ ]:
for job_div in job_div_tag_list:
    a_tag = job_div.find('a')
if a_tag:
        inst_name = a_tag.text.strip()
        job_link = base_url + '/' + a_tag['href'].strip()

        try:
          job_response = requests.get(job_link)
          job_soup = bs(job_response.content, 'html.parser')
          details = job_soup.get_text(separator=' \n')

          def extract_between(text, start, end):
            try:
              return text.split(start)[1].split(end)[0]
            except:
              return ''

          pos_id = extract_between(details, 'Job ID:', '\n')
          pos_title = extract_between(details, 'Position Title:', '\n')
          location = extract_between(details, 'Location:', '\n')
          subject = extract_between(details, 'Subject Area:', '\n')
          deadline = extract_between(details, 'Application Deadline:', '\n')

          apply_a = job_soup.find('a', text='Apply')
          apply_link = base_url + apply_a['href'] if apply_a and 'href' in apply_a.attrs else ''

        except Exception as ex:
          print(f'Error processing {job_link} - {ex}')
          pos_id = pos_title = location = subject = deadline = apply_link = ''

        for key in data.keys():
          if key == 'Institution Name':
            data[key].append(inst_name)
          elif key == 'Position ID':
            data[key].append(pos_id)
          elif key == 'Position Title':
            data[key].append(pos_title)
          elif key == 'Position Location':
            data[key].append(location)
          elif key == 'Subject Area':
            data[key].append(subject)
          elif key == 'Application Deadline':
            data[key].append(deadline)
          elif key == 'Apply Link':
            data[key].append(apply_link)
          elif key == 'Job Details Link':
            data[key].append(job_link)

<ipython-input-6-b6ddb75fb2cd>:24: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  apply_a = job_soup.find('a', text='Apply')


In [ ]:
df = pd.DataFrame(data)
df.to_csv('mathjobs.csv', index=False)

print("Scraping complete. Data saved to 'mathjobs.cvs'.")

Scraping complete. Data saved to 'mathjobs.cvs'.
